In [2]:
import pandas as pd
import re
import json
from tqdm import tqdm

from llama_index.llms.openai import OpenAI

from dotenv import load_dotenv
import os
load_dotenv()

True

# Build dictionary of slangs

- Slangs: https://www.kaggle.com/code/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing/comments
- Singlish: https://thehoneycombers.com/singapore/singlish-101/

In [2]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nothing",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired",

    "act blur": "act ignorant",
    "agak agak": "rough estimate",
    "aiyoh": "An expression of surprise, and/or annoyance",
    "alamak": "oh my gosh",
    "arrow": "to dump a task on someone else, rather than complete it yourself",
    "atas": "to be posh or of high social status",
    "bo chap": "don't care in Hokkien, referring to someone being indifferent",
    "bo jio": "Hokkien for not getting an invitation",
    "bo liao": "feeling bored or idle like there's nothing better to do",
    "bo leh": "a malay word for 'can', or 'possible'",
    "can or not": "a way of asking if something is possible or can be achieved",
    "can a not": "a way of asking if something is possible or can be achieved",
    "catch no ball": "to be absolutely clueless",
    "cheem": "something that has you dumbfounded, perplexed, bewildered or confused",
    "chiong": "to rush, to hurry, or to give your all to complete something",
    "chope": "to reserve a place or call dibs on something",
    "cmi": "the abbreviation for “cannot make it” refers to the inferior attributes of someone (or something)",
    "come, I clap for you": "a sarcastic way of praising someone",
    "diam": "used to tell someone to shut up, typically in an angry way",
    "die die must": "possibly to express something that is so good",
    "eh": "a way to address people or get their attention",
    "eye power": "someone who does not extend help. Instead, they just stand around and stare as if their eyes can offer assistance",
    "geh kiang": "someone who acts rashly and without thought",
    "gong": "crazy",
    "huat": "to prosper",
    "jia lat": "nothing is going your way",
    "kaypoh": "to be a busybody",
    "kpoh": "to be a busybody",
    "kena": "to describe being affected by or to receive something negative",
    "kiasu": "a fiercely competitive spirit in a negative connotation",
    "kiasi": "scared to death or too afraid of failing",
    "lah": "a suffix used to emphasize the sentence or word before",
    "leh": "use 'leh' when something is unsure---it's more like a question",
    "lepak": "chilling without a care in the world or loitering around aimlessly",
    "lobang": "tips, clues, opportunities or deals",
    "lor": "it holds a sense of resignation and finality",
    "makan": "eat",
    "nia": "used to substitute only, often used to belittle something or someone",
    "own time own target": "to do things at your own pace",
    "otot": "to do things at your own pace",
    "paiseh": "embarrassed or shy",
    "pai seh": "embarrassed or shy",
    "pang seh": "to stand someone up or to cop out",
    "ponteng": "playing truant, but it can be used if you want to give anything a miss",
    "sabo": "sabotage",
    "shag": "to feel tired or exhausted",
    "shiok": "fantastic or to convey feelings of satisfaction and pleasure",
    "sian": "experiencing boredom, a lack of enthusiasm or just being tired of life",
    "sian": "crazy",
    "spoil market": "a singlish term for overachieving",
    "steady": "it can take on different meanings, including agreeing with someone, giving them praise or being in a romantic relationship with a partner",
    "swee": "a compliment to describe something as beautiful or perfect",
    "tak boleh tahan": "something is intolerable",
    "buay tahan": "something is intolerable",
    "tapao": "takeaway or failing a subject",
    "dapao": "takeaway or failing a subject",
    "tolong": "a cry for help",
    "ulu": "somewhere secluded and inaccessible",
    "walao": "an expression of shock, disbelief or dismay",
    "wayang": "used to describe someone who's being fake",
    "White horse": "the child of an influential or powerful person (usually used among men in National Service)",
    "ns": "national service where boys serve the army at the age of 18 for two years",
    "NS": "national service where boys serve the army at the age of 18 for two years",
    "zhng": "to modify, embellish, or redecorate",
    "zng": "to modify, embellish, or redecorate",
}



- Paper: This Word Mean What: Constructing a Singlish Dictionary with ChatGPT, https://aclanthology.org/2024.eurali-1.7.pdf
- GitHub: https://github.com/singdict/SingDict

In [ ]:
folder_path = "../../../SingDict/vocabulary" # see github link above

merged_dict = {}

# Loop through all JSONL files in the folder
for file_name in sorted(os.listdir(folder_path)):
    if file_name.endswith(".jsonl"):
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                json_data = json.loads(line.strip())

                word = json_data["Word"]
                Definition = json_data["Definition"]

                merged_dict[word] = Definition

In [ ]:
# Merge both dictionaries
upd_abbreviations = {**abbreviations, **merged_dict}

{'$': ' dollar ', '€': ' euro ', '4ao': 'for adults only', 'a.m': 'before midday', 'a3': 'anytime anywhere anyplace', 'aamof': 'as a matter of fact', 'acct': 'account', 'adih': 'another day in hell', 'afaic': 'as far as i am concerned', 'afaict': 'as far as i can tell', 'afaik': 'as far as i know', 'afair': 'as far as i remember', 'afk': 'away from keyboard', 'app': 'application', 'approx': 'approximately', 'apps': 'applications', 'asap': 'as soon as possible', 'asl': 'age, sex, location', 'atk': 'at the keyboard', 'ave.': 'avenue', 'aymm': 'are you my mother', 'ayor': 'at your own risk', 'b&b': 'bed and breakfast', 'b+b': 'bed and breakfast', 'b.c': 'before christ', 'b2b': 'business to business', 'b2c': 'business to customer', 'b4': 'before', 'b4n': 'bye for now', 'b@u': 'back at you', 'bae': 'before anyone else', 'bak': 'A popular Hokkien word for meat used in Singlish.', 'bbbg': 'bye bye be good', 'bbc': 'british broadcasting corporation', 'bbias': 'be back in a second', 'bbl': 'be 

## Save this dictionary of slangs

In [ ]:
folder_path = "dictionary"
json_file_path = os.path.join(folder_path, "slang.json")

os.makedirs(folder_path, exist_ok=True)

with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(upd_abbreviations, f, indent=4, ensure_ascii=False)

print(f"Dictionary saved to {json_file_path}")

Dictionary saved to dictionary/slang.json


# Round 1 (Facebook Comments)

## Load slang dictionary

In [ ]:
json_file_path = "dictionary/slang.json"

with open(json_file_path, "r", encoding="utf-8") as f:
    slang_dict = json.load(f)

## Load facebook comments

In [ ]:
facebook_df = pd.read_excel("data/en_dataset.xlsx")
Available_emotions = list(facebook_df.emotion.unique())
Available_emotions

['ANGER', 'DISGUST', 'FEAR', 'JOY', 'SADNESS', 'SURPRISE', 'NEUTRAL']

## Init worker

In [8]:
myWorker = OpenAI(model="gpt-4o-mini", temperature=0, max_retries=20)

## Prompt

In [16]:
def slang_prompt(comment, entity, emotion, Available_emotions, slang_part):
    prompt = f"""# Objective: You are an expert annotator. Your task is to extract the aspect, opinion, and sentiment from the comment below based on the provided entity and emotion.

## Rules:
1. When assigning the Emotion choose one from: {Available_emotions}.
2. When assigning the Sentiment choose one from: ['POSITIVE', 'NEGATIVE', 'NEUTRAL'].
3. Do not adjust the provided Entity and Emotion.
4. If the comment contains slang, **use the provided meaning to infer the correct aspect and opinion**.
5. If the slang expresses sentiment incorporate it into the opinion.
6. Always use this structure:
```
Comment:
Entity:
Aspect:
Opinion:
Emotion:
Sentiment:
```

## Examples:

### Example 1:
Comment: "That movie was straight fire!🔥 Best thing I've watched all year."
Entity: Movie
Aspect: Quality
Opinion: "straight fire (amazing quality)"
Emotion: JOY
Sentiment: POSITIVE
Slang: "fire" → "amazing"

### Example 2:
Comment: "Bro, this queue is mad long. I'm losing my mind."
Entity: Queue
Aspect: Wait Time
Opinion: "mad long (very long)"
Emotion: ANGER
Sentiment: NEGATIVE
Slang: "mad long" → "very long"

# Task:
Comment: {comment}
Entity: {entity}
Aspect:
Opinion:
Emotion: {emotion}
Sentiment:
{slang_part}
"""
    return prompt


## Processing

In [ ]:
start_index = 0

resp_list = []

for index, row in tqdm(facebook_df.iloc[start_index:].iterrows(), total=len(facebook_df) - start_index):
    # Extract values from the row
    comment = row.text
    entity = row.entity
    emotion = row.emotion

    # Collect slang meanings
    pot_slangs = comment.split()
    slang_part = ""
    for slang in pot_slangs:
        slang = re.sub(r"[^\w\s]", "", slang.strip().lower())
        if slang in abbreviations:
            slang_part += f"\nAdditional information: The slang '{slang}' in the comment means '{abbreviations[slang]}'. Use this meaning to refine the aspect, opinion and sentiment."

    # Generate prompt including all detected slang meanings
    s_prompt = slang_prompt(comment, entity, emotion, Available_emotions, slang_part)

    # Get model response
    resp = myWorker.complete(s_prompt)

    # Parse the response safely
    lines = resp.text.split("\n")
    annotation = {line.split(": ")[0]: line.split(": ")[1] for line in lines if ": " in line}

    resp_list.append(annotation)

100%|██████████| 2571/2571 [1:27:52<00:00,  2.05s/it]  


## Saving the raw responses

In [ ]:
folder_path = "backup_v2"
json_file_path = os.path.join(folder_path, "fb_resp_list-GPT4o_mini.json")

os.makedirs(folder_path, exist_ok=True)

with open(json_file_path, "w") as file:
    json.dump(resp_list, file, indent=4)

In [37]:
fb_GPT4omini = pd.DataFrame(resp_list)
new_facebook_df = pd.DataFrame(
    {
        'text': facebook_df.text,
        'entity': facebook_df.entity,
        'emotion': facebook_df.emotion,
        'aspect': fb_GPT4omini.Aspect,
        'opinion': fb_GPT4omini.Opinion,
        'sentiment': fb_GPT4omini.Sentiment
    }
)

## Demojize and stripping of leading and trailing spaces

In [38]:
import emoji

cols_to_strip = ["aspect", "opinion", "sentiment"]
new_facebook_df[cols_to_strip] = new_facebook_df[cols_to_strip].apply(lambda x: x.str.strip())
new_facebook_df['text'] = new_facebook_df['text'].apply(emoji.demojize)

## Filter and move invalid rows

In [42]:
invalid_values = [None, "", "Unknown", "N/A", "None"]

mask = new_facebook_df[["aspect", "opinion", "sentiment"]].apply(lambda x: x.isin(invalid_values)).any(axis=1)

fb_df_invalid = new_facebook_df[mask]  # Rows to be moved
fb_df_cleaned = new_facebook_df[~mask]  # Cleaned DataFrame

fb_df_invalid.reset_index(drop=True, inplace=True)
fb_df_cleaned.reset_index(drop=True, inplace=True)

## Saving cleaned data

In [43]:
fb_df_invalid.to_excel("./backup_v2/fb_df_invalid.xlsx", index=False)
fb_df_cleaned.to_excel("./backup_v2/fb_df_cleaned.xlsx", index=False)

## Findings

Without the slang dictionary, GPT-4o-mini may extract wrongly. 

### Without the slang dictionary
row 271
- Comment: WP sucks! PAP ftw!
- Entity: WP
- Emotion: DISGUST
- Aspect: Performance
- Opinion: "sucks"
- Sentiment: NEGATIVE

row 272
- Comment: WP sucks! PAP ftw!
- Entity: PAP
- Emotion: JOY
- Aspect: Performance
- Opinion: "sucks"
- Sentiment: NEGATIVE

### With the slang dictionary
row 271
- Comment: WP sucks! PAP ftw!
- Entity: WP
- Emotion: DISGUST
- Aspect: Performance
- Opinion: "sucks"
- Sentiment: NEGATIVE

row 272
- Comment: WP sucks! PAP ftw!
- Entity: PAP
- Emotion: JOY
- Aspect: Performance
- Opinion: "ftw (for the win)"
- Sentiment: POSITIVE

# Round 2 (Reddit Comments)

## Load slang dictionary

In [ ]:
json_file_path = "dictionary/slang.json"

with open(json_file_path, "r", encoding="utf-8") as f:
    slang_dict = json.load(f)

## Load Reddit comments and Emotions from Facebook comments

In [ ]:
facebook_df = pd.read_excel("data/en_dataset.xlsx")
Available_emotions = list(facebook_df.emotion.unique())

reddit_df = pd.read_csv("data/reddit_comments.csv")

## Init Worker

In [124]:
myWorker = OpenAI(model="gpt-4o-mini", temperature=0, max_retries=20)

## Prompt

In [125]:
def reddit_prompt_updates(comment, Available_emotions, slang_part):
    prompt = f"""# Objective: You You are an expert annotator. Your task is to extract Entity, Aspect, Opinion, Emotion, and Sentiment from the given comment. Each comment can have multiple entities and aspects, return a structured JSON output.

## Rules:
1. Each entity must be associated with at least one aspect, opinion, emotion, and sentiment.
2. Emotion must be one of: {Available_emotions}.
3. Sentiment must be one of: ['POSITIVE', 'NEGATIVE', 'NEUTRAL'].
4. If a comment does not explicitly mention an entity, aspect, opinion, emotion, or sentiment, do not infer missing details. Leave those fields blank.
5. Format the response as a **JSON list**, where each entity is a dictionary.
```
[
    {{
        "Entity": "...",
        "Aspect": "...",
        "Opinion": "...",
        "Emotion": "...",
        "Sentiment": "..."
    }},
    {{...}},
    ...
]
```

## Examples:

### Example 1:
Comment: "Starbucks coffee is overpriced but their atmosphere is cozy."
[
    {{
        "Entity": "Starbucks",
        "Aspect": "Price",
        "Opinion": ""overpriced"",
        "Emotion": "DISGUST",
        "Sentiment": "NEGATIVE"
    }},
    {{
        "Entity": "Starbucks",
        "Aspect": "Atmosphere",
        "Opinion": "cozy",
        "Emotion": "SURPRISE",
        "Sentiment": "POSITIVE"
    }}
]

### Example 2:
Comment: "usa so kpoh for what? biden own country like slumps still wan kpoh?"
[
    {{
        "Entity": "USA",
        "Aspect": "Foreign Policy",
        "Opinion": "kpoh (busybody)",
        "Emotion": "DISGUST",
        "Sentiment": "NEGATIVE"
    }},
    {{
        "Entity": "Biden",
        "Aspect": "Governance",
        "Opinion": "own country like slumps",
        "Emotion": "DISGUST",
        "Sentiment": "NEGATIVE"
    }}
]

### Example 10:
Comment: "shiok sia! ytd woodlands and JB customs no queue at all"
[
    {{
        "Entity": "Woodlands",
        "Aspect": "Wait Time",
        "Opinion": "no queue at all",
        "Emotion": "JOY",
        "Sentiment": "POSITIVE"
    }},
    {{
        "Entity": "JB customs",
        "Aspect": "Wait Time",
        "Opinion": ""no queue at all"",
        "Emotion": "JOY",
        "Sentiment": "POSITIVE"
    }}
]

# Task:
Comment: "{comment}"
{slang_part}

"""
    return prompt

## Process

In [ ]:
start_index = 0  # row number to start from
reddit_list = []

for index, row in tqdm(reddit_df.iloc[start_index:].iterrows(), total=len(reddit_df) - start_index):
    comment = row.comment

    # Collect slang meanings
    pot_slangs = comment.split()
    slang_part = ""
    for slang in pot_slangs:
        slang = re.sub(r"[^\w\s]", "", slang.strip().lower())
        if slang in upd_abbreviations:
            slang_part += f"\nAdditional information: The slang '{slang}' in the comment could mean '{upd_abbreviations[slang]}'. Use this potential meaning to refine the aspect, opinion and sentiment."

    r_prompt = reddit_prompt_updates(comment, Available_emotions, slang_part)

    response = myWorker.complete(r_prompt)
    cleaned_response = response.text.strip("```json").strip("```").strip()
    reddit_list.append(cleaned_response)

100%|██████████| 3589/3589 [2:08:17<00:00,  2.14s/it]  


## Saving the raw responses

In [ ]:
folder_path = "backup_v2"
json_file_path = os.path.join(folder_path, "raw_reddit_list-GPT4o_mini.json")

os.makedirs(folder_path, exist_ok=True)

# Save to JSON file
with open(json_file_path, "w") as file:
    json.dump(reddit_list, file, indent=4)

## Make the responses into a dataframe

In [18]:
with open("backup_v2/raw_reddit_list-GPT4o_mini.json", "r", encoding="utf-8") as f:
    loaded_reddit = json.load(f)

In [19]:
processed_list = []

for cleaned_replies, original in zip(loaded_reddit, reddit_df.itertuples(index=False)):
    comment = original.comment

    try:
        annotations = json.loads(cleaned_replies)
    except json.JSONDecodeError:
        print(f"Error parsing JSON")
        print(original)
        break

    for annotation in annotations:
        annotation["text"] = comment  # Duplicate the comment for each annotation
        processed_list.append(annotation)

In [20]:
reddit_GPT4omini = pd.DataFrame(processed_list)
reddit_GPT4omini.rename(columns={"Entity": "entity", "Aspect": "aspect", "Emotion": "emotion", "Opinion": "opinion", "Sentiment": "sentiment"}, inplace=True)
new_order = ["text", "entity", "emotion", "aspect", "opinion", "sentiment"]
reddit_GPT4omini = reddit_GPT4omini[new_order]

## Demojize and stripping of leading and trailing spaces

In [21]:
import emoji

cols_to_strip = ["aspect", "opinion", "sentiment"]
reddit_GPT4omini[cols_to_strip] = reddit_GPT4omini[cols_to_strip].apply(lambda x: x.str.strip())
reddit_GPT4omini['text'] = reddit_GPT4omini['text'].apply(lambda x: emoji.demojize(str(x)) if isinstance(x, str) else "") # Replaces NaN or floats with "" to avoid errors

## Map new emotions to available 7

In [ ]:
emotion_mapping = {
    "HOPE": "JOY",
    "CONFUSION": "SURPRISE",
    "FRUSTRATION": "ANGER",
    "ANGRY": "ANGER",
    "DISAPPOINTMENT": "SADNESS",
    "FAIRNESS": "NEUTRAL",
    "DISHEARTENING": "SADNESS",
    "ANNOYANCE": "ANGER",
    "EMPATHY": "NEUTRAL",
    "SHAME": "DISGUST",
    "PUZZLED": "SURPRISE",
    "CURIOSITY": "SURPRISE",
    "ANGER": "ANGER",
    "DISGUST": "DISGUST",
    "FEAR": "FEAR",
    "JOY": "JOY",
    "SADNESS": "SADNESS",
    "SURPRISE": "SURPRISE",
    "NEUTRAL": "NEUTRAL"
}

reddit_GPT4omini["emotion"] = reddit_GPT4omini["emotion"].map(emotion_mapping)

## Remove Illegal Characters Before Saving

In [24]:
def remove_illegal_chars(text):
    if isinstance(text, str):  # Ensure it's a string
        return re.sub(r'[\x00-\x1F\x7F]', '', text)  # Remove control characters
    return text  # Return unchanged for non-strings

new_reddit_df = reddit_GPT4omini.map(remove_illegal_chars)

## Filter and move invalid rows

In [25]:
invalid_values = [None, "", "Unknown", "N/A", "None"]

mask = new_reddit_df[["text", "entity", "emotion", "aspect", "opinion", "sentiment"]].apply(lambda x: x.isin(invalid_values)).any(axis=1)

reddit_df_invalid = new_reddit_df[mask]  # Rows to be moved
reddit_df_cleaned = new_reddit_df[~mask]  # Cleaned DataFrame

reddit_df_invalid.reset_index(drop=True, inplace=True)
reddit_df_cleaned.reset_index(drop=True, inplace=True)

## Saving cleaned data

In [26]:
import openpyxl

reddit_df_invalid.to_excel("backup_v2/reddit_df_invalid.xlsx", index=False, engine="openpyxl")
reddit_df_cleaned.to_excel("backup_v2/reddit_df_cleaned.xlsx", index=False, engine="openpyxl")

# Creating Train-Validation set and Test Set

In [27]:
fb_cleaned = pd.read_excel("backup_v2/fb_df_cleaned.xlsx")
reddit_cleaned = pd.read_excel("backup_v2/reddit_df_cleaned.xlsx")

In [28]:
df_combined = pd.concat([fb_cleaned, reddit_cleaned], ignore_index=True)
print(df_combined["emotion"].value_counts(normalize=True))
print(df_combined["emotion"].value_counts())

emotion
NEUTRAL     0.229758
DISGUST     0.202081
JOY         0.188472
SADNESS     0.134035
ANGER       0.111619
FEAR        0.079140
SURPRISE    0.054895
Name: proportion, dtype: float64
emotion
NEUTRAL     2009
DISGUST     1767
JOY         1648
SADNESS     1172
ANGER        976
FEAR         692
SURPRISE     480
Name: count, dtype: int64


In [29]:
from sklearn.model_selection import train_test_split

# 80-20 split first (train-validation + test)
train_val_df, test_df = train_test_split(
    df_combined,
    test_size=0.20,
    stratify=df_combined["emotion"],
    random_state=42
)


In [30]:
train_val_df.to_excel("backup_v2/train_val_df.xlsx", index=False, engine="openpyxl")
test_df.to_excel("backup_v2/test_df.xlsx", index=False, engine="openpyxl")

In [31]:
len(df_combined)

8744